## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [2]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the enviro    nment variable

groq_api_key=os.getenv("GROQ_API_KEY")



In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000026098EF2A40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000026098EF3700>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Sahu and I am a Chief data Scientist")])

AIMessage(content="Hello Sahu, it's nice to meet you!\n\nIt's great to have a Chief Data Scientist on board.  What can I do for you today? Are you looking for help with a specific data science problem, or just want to chat about the field?\n\nI can assist with things like:\n\n* **Generating code:** I can write code in various programming languages, including Python and R, for data manipulation, analysis, and modeling.\n* **Explaining concepts:**  I can explain data science concepts, algorithms, and techniques in a clear and concise way.\n* **Brainstorming ideas:**  I can help you brainstorm new ideas for data science projects or solutions to existing problems.\n* **Staying up-to-date:**  I can provide you with information on the latest trends and developments in the field of data science.\n\n\nLet me know how I can be of service! \n\n", response_metadata={'token_usage': {'completion_tokens': 191, 'prompt_tokens': 28, 'total_tokens': 219, 'completion_time': 0.400763872, 'prompt_time': 0

In [5]:
from langchain_core.messages import AIMessage
# Passing set of human and AI messages in the form of list
model.invoke(
    [
        HumanMessage(content="Hi , My name is Sahu and I am a Chief AI Engineer"),
        AIMessage(content="Hello Sahu! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Sahu, and you told me you are a Chief AI Engineer!  \n\nIs there anything else you'd like me to remember about you? 😊  Perhaps your favorite AI project or the company you work for?  The more I know, the better I can interact with you. \n\n", response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 104, 'total_tokens': 169, 'completion_time': 0.1341055, 'prompt_time': 0.006431094, 'queue_time': None, 'total_time': 0.140536594}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-db336b32-10c6-4aad-bb24-6922b1a38b88-0', usage_metadata={'input_tokens': 104, 'output_tokens': 65, 'total_tokens': 169})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
# !pip install langchain_community

In [7]:
# Chat message history
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory 
from langchain_core.runnables.history import RunnableWithMessageHistory # For chains

In [12]:
store={} # To store the session_id
# To separate the sessions from one another
# We have input for as the session_id: str for individual chat
# It will return a BaseChatHistory  
def get_session_history(session_id:str)->BaseChatMessageHistory:
    print("Store: ",store)
    if session_id not in store:
        store[session_id]=ChatMessageHistory() # Chat should go in the session_id
    return store[session_id]


In [13]:
with_message_history=RunnableWithMessageHistory(model,get_session_history)
with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
| RunnableBranch(branches=[(RunnableBinding(bound=RunnableLambda(_is_not_async), config={'run_name': 'RunnableWithMessageHistoryInAsyncMode'}), RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000026098EF2A40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000026098EF3700>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********')), config_factories=[<function Runnable.with_alisteners.<locals>.<lambda> at 0x00000260990D0940>]))], default=RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000026098EF2A40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000026098EF3700>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********')), config_factories=[<function Runnable.with_listeners.<lo

In [14]:
config={"configurable":{"session_id":"chat1"}} # Giving our own session_id but not required

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Sahu and I am a Chief AI Engineer")],
    config=config # passing the config for given session_id of chat1
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


Store:  {}


In [18]:
response.content

"Hello Sahu,\n\nIt's nice to meet you! Being a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on? \n\nI'm eager to learn more about your work in the field of AI.\n"

In [16]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


Store:  {'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is Sahu and I am a Chief AI Engineer'), AIMessage(content="Hello Sahu,\n\nIt's nice to meet you! Being a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on? \n\nI'm eager to learn more about your work in the field of AI.\n", response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 28, 'total_tokens': 82, 'completion_time': 0.111580355, 'prompt_time': 0.00265645, 'queue_time': None, 'total_time': 0.11423680500000001}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bed19a4c-7b27-4196-ba58-d590162dd53c-0', usage_metadata={'input_tokens': 28, 'output_tokens': 54, 'total_tokens': 82})])}


AIMessage(content='Your name is Sahu. 😊  \n\nYou told me at the beginning of our conversation!  \n\nIs there anything else I can help you with?\n', response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 96, 'total_tokens': 130, 'completion_time': 0.069427948, 'prompt_time': 0.004551999, 'queue_time': None, 'total_time': 0.073979947}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9f735a24-8f85-402a-b001-9cd219e31e6e-0', usage_metadata={'input_tokens': 96, 'output_tokens': 34, 'total_tokens': 130})

In [17]:
with_message_history.invoke(
    [HumanMessage(content="What's my job?")],
    config=config,
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


Store:  {'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is Sahu and I am a Chief AI Engineer'), AIMessage(content="Hello Sahu,\n\nIt's nice to meet you! Being a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on? \n\nI'm eager to learn more about your work in the field of AI.\n", response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 28, 'total_tokens': 82, 'completion_time': 0.111580355, 'prompt_time': 0.00265645, 'queue_time': None, 'total_time': 0.11423680500000001}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bed19a4c-7b27-4196-ba58-d590162dd53c-0', usage_metadata={'input_tokens': 28, 'output_tokens': 54, 'total_tokens': 82}), HumanMessage(content="What's my name?"), AIMessage(content='Your name is Sahu. 😊  \n\nYou told me at the beginning of our conversation!  \n\nIs there anything else I can help you with?\n'

AIMessage(content='You said you are a Chief AI Engineer.  \n\nIs there anything else I can help you with regarding your work or anything else at all?\n', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 144, 'total_tokens': 176, 'completion_time': 0.06538629, 'prompt_time': 0.007101069, 'queue_time': None, 'total_time': 0.072487359}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-13ef99cd-521c-4233-b33e-d988434f4114-0', usage_metadata={'input_tokens': 144, 'output_tokens': 32, 'total_tokens': 176})

In [19]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

Store:  {'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is Sahu and I am a Chief AI Engineer'), AIMessage(content="Hello Sahu,\n\nIt's nice to meet you! Being a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on? \n\nI'm eager to learn more about your work in the field of AI.\n", response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 28, 'total_tokens': 82, 'completion_time': 0.111580355, 'prompt_time': 0.00265645, 'queue_time': None, 'total_time': 0.11423680500000001}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bed19a4c-7b27-4196-ba58-d590162dd53c-0', usage_metadata={'input_tokens': 28, 'output_tokens': 54, 'total_tokens': 82}), HumanMessage(content="What's my name?"), AIMessage(content='Your name is Sahu. 😊  \n\nYou told me at the beginning of our conversation!  \n\nIs there anything else I can help you with?\n'

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'As an AI, I have no memory of past conversations and do not know your name. Please tell me your name so I can address you properly.\n'

In [20]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Sahu")],
    config=config1
)
response.content

Store:  {'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is Sahu and I am a Chief AI Engineer'), AIMessage(content="Hello Sahu,\n\nIt's nice to meet you! Being a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on? \n\nI'm eager to learn more about your work in the field of AI.\n", response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 28, 'total_tokens': 82, 'completion_time': 0.111580355, 'prompt_time': 0.00265645, 'queue_time': None, 'total_time': 0.11423680500000001}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bed19a4c-7b27-4196-ba58-d590162dd53c-0', usage_metadata={'input_tokens': 28, 'output_tokens': 54, 'total_tokens': 82}), HumanMessage(content="What's my name?"), AIMessage(content='Your name is Sahu. 😊  \n\nYou told me at the beginning of our conversation!  \n\nIs there anything else I can help you with?\n'

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"Hello Sahu! It's nice to meet you. \n\nWhat can I do for you today? 😊  \n\n"

In [21]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()


Store:  {'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is Sahu and I am a Chief AI Engineer'), AIMessage(content="Hello Sahu,\n\nIt's nice to meet you! Being a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on? \n\nI'm eager to learn more about your work in the field of AI.\n", response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 28, 'total_tokens': 82, 'completion_time': 0.111580355, 'prompt_time': 0.00265645, 'queue_time': None, 'total_time': 0.11423680500000001}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bed19a4c-7b27-4196-ba58-d590162dd53c-0', usage_metadata={'input_tokens': 28, 'output_tokens': 54, 'total_tokens': 82}), HumanMessage(content="What's my name?"), AIMessage(content='Your name is Sahu. 😊  \n\nYou told me at the beginning of our conversation!  \n\nIs there anything else I can help you with?\n'

Error in callback coroutine: ValueError()


'Your name is Sahu!  \n\nI remember that you told me earlier. 😄  How can I help you further?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [22]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the question to the nest of your ability"),
        # Now while passing the message we need to give it ina key value pair
        MessagesPlaceholder(variable_name="messages") # Key: is messages
    ]
)

chain=prompt|model

In [23]:
# Passing the human message in the form of dictionary
chain.invoke({"messages":[HumanMessage(content="Hi My name is Sahu")]})

AIMessage(content="Hi Sahu, it's nice to meet you! 👋 \n\nWhat can I do for you today? 😊  \n", response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 36, 'total_tokens': 64, 'completion_time': 0.056709443, 'prompt_time': 0.00265473, 'queue_time': None, 'total_time': 0.059364173}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-03ef1e57-de0e-41d0-bcf6-f73e2aa0a67d-0', usage_metadata={'input_tokens': 36, 'output_tokens': 28, 'total_tokens': 64})

In [25]:
# This will create message history
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [26]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Sahu")],
    config=config
)

response

Store:  {'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is Sahu and I am a Chief AI Engineer'), AIMessage(content="Hello Sahu,\n\nIt's nice to meet you! Being a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on? \n\nI'm eager to learn more about your work in the field of AI.\n", response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 28, 'total_tokens': 82, 'completion_time': 0.111580355, 'prompt_time': 0.00265645, 'queue_time': None, 'total_time': 0.11423680500000001}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bed19a4c-7b27-4196-ba58-d590162dd53c-0', usage_metadata={'input_tokens': 28, 'output_tokens': 54, 'total_tokens': 82}), HumanMessage(content="What's my name?"), AIMessage(content='Your name is Sahu. 😊  \n\nYou told me at the beginning of our conversation!  \n\nIs there anything else I can help you with?\n'

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


AIMessage(content="Hello Sahu, it's nice to meet you! 👋\n\nIs there anything I can help you with today? 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 36, 'total_tokens': 66, 'completion_time': 0.060593562, 'prompt_time': 0.002587263, 'queue_time': None, 'total_time': 0.063180825}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e87a44d4-4fc3-4e36-90b1-d5d0368c9b12-0', usage_metadata={'input_tokens': 36, 'output_tokens': 30, 'total_tokens': 66})

In [27]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Store:  {'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is Sahu and I am a Chief AI Engineer'), AIMessage(content="Hello Sahu,\n\nIt's nice to meet you! Being a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on? \n\nI'm eager to learn more about your work in the field of AI.\n", response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 28, 'total_tokens': 82, 'completion_time': 0.111580355, 'prompt_time': 0.00265645, 'queue_time': None, 'total_time': 0.11423680500000001}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bed19a4c-7b27-4196-ba58-d590162dd53c-0', usage_metadata={'input_tokens': 28, 'output_tokens': 54, 'total_tokens': 82}), HumanMessage(content="What's my name?"), AIMessage(content='Your name is Sahu. 😊  \n\nYou told me at the beginning of our conversation!  \n\nIs there anything else I can help you with?\n'

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'Your name is Sahu.  😊 \n\nI remember!  How can I help you further?\n'

In [28]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [30]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Sahu")],"language":"French"})
response.content

"Bonjour Sahu, enchanté de faire votre connaissance ! Comment puis-je vous aider aujourd'hui ? 😊  \n"

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [31]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history, # A dict with a key for a BaseMessage or sequence of BaseMessages
    input_messages_key="messages" # Must be specified if the base runnable accepts a dict as input. Default is None.
)

In [34]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Sahu")],"language":"French"},
    config=config
)
repsonse.content

Store:  {'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is Sahu and I am a Chief AI Engineer'), AIMessage(content="Hello Sahu,\n\nIt's nice to meet you! Being a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on? \n\nI'm eager to learn more about your work in the field of AI.\n", response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 28, 'total_tokens': 82, 'completion_time': 0.111580355, 'prompt_time': 0.00265645, 'queue_time': None, 'total_time': 0.11423680500000001}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bed19a4c-7b27-4196-ba58-d590162dd53c-0', usage_metadata={'input_tokens': 28, 'output_tokens': 54, 'total_tokens': 82}), HumanMessage(content="What's my name?"), AIMessage(content='Your name is Sahu. 😊  \n\nYou told me at the beginning of our conversation!  \n\nIs there anything else I can help you with?\n'

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"Bonjour Sahu !  Enchanté de faire votre connaissance. 😊 Comment puis-je vous aider aujourd'hui ? \n"

In [36]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

Store:  {'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is Sahu and I am a Chief AI Engineer'), AIMessage(content="Hello Sahu,\n\nIt's nice to meet you! Being a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on? \n\nI'm eager to learn more about your work in the field of AI.\n", response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 28, 'total_tokens': 82, 'completion_time': 0.111580355, 'prompt_time': 0.00265645, 'queue_time': None, 'total_time': 0.11423680500000001}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bed19a4c-7b27-4196-ba58-d590162dd53c-0', usage_metadata={'input_tokens': 28, 'output_tokens': 54, 'total_tokens': 82}), HumanMessage(content="What's my name?"), AIMessage(content='Your name is Sahu. 😊  \n\nYou told me at the beginning of our conversation!  \n\nIs there anything else I can help you with?\n'

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


In [37]:
response.content

'आपला नाव Sahu आहे.  \n'

In [41]:
store['chat4'].messages

[HumanMessage(content='Hi,I am Sahu'),
 AIMessage(content='नमस्कार, साहू! मी तुमच्या मदतीसाठी येथे आहे. काय मदत करू शकतो? \n', response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 38, 'total_tokens': 73, 'completion_time': 0.071320707, 'prompt_time': 0.002883445, 'queue_time': None, 'total_time': 0.074204152}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f7196b67-2c95-4a68-8100-c3b834fb347f-0', usage_metadata={'input_tokens': 38, 'output_tokens': 35, 'total_tokens': 73}),
 HumanMessage(content='Hi,I am Sahu'),
 AIMessage(content='नमस्कार, साहू! 🎉 \n\nमी तुम्हाला मदत करण्यासाठी उत्सुक आहे. काय करू शकतो? 🤔\n', response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 88, 'total_tokens': 128, 'completion_time': 0.082382122, 'prompt_time': 0.004691591, 'queue_time': None, 'total_time': 0.087073713}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'fini

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

`trim_messages` helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [42]:
from langchain_core.messages import SystemMessage,trim_messages

In [44]:
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

u:\Langchain-repo\.genai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
u:\Langchain-repo\.genai\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\utkar\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [45]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough
 
chain=(
    # Passing the trimmer func in the chain
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a large language model, I don't have access to your personal information, including your ice cream preferences.\n\nWhat's your favorite flavor?  🍦\n"

In [46]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "What\'s 2 + 2?".  \n\n\n\n\n'

In [47]:
chain

RunnableAssign(mapper={
  messages: RunnableLambda(itemgetter('messages'))
            | RunnableLambda(...)
})
| ChatPromptTemplate(input_variables=['language', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], template='You are a helpful assistant. Answer all questions to the best of your ability in {language}.')), MessagesPlaceholder(variable_name='messages')])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000026098EF2A40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000026098EF3700>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [48]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [49]:
# This is checking it in the chat5 session_id which is empty
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

Store:  {'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is Sahu and I am a Chief AI Engineer'), AIMessage(content="Hello Sahu,\n\nIt's nice to meet you! Being a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on? \n\nI'm eager to learn more about your work in the field of AI.\n", response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 28, 'total_tokens': 82, 'completion_time': 0.111580355, 'prompt_time': 0.00265645, 'queue_time': None, 'total_time': 0.11423680500000001}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bed19a4c-7b27-4196-ba58-d590162dd53c-0', usage_metadata={'input_tokens': 28, 'output_tokens': 54, 'total_tokens': 82}), HumanMessage(content="What's my name?"), AIMessage(content='Your name is Sahu. 😊  \n\nYou told me at the beginning of our conversation!  \n\nIs there anything else I can help you with?\n'

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"I don't have access to any information about you, including your name. If you'd like to tell me, I'd be happy to know!\n"

In [50]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()


Store:  {'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is Sahu and I am a Chief AI Engineer'), AIMessage(content="Hello Sahu,\n\nIt's nice to meet you! Being a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on? \n\nI'm eager to learn more about your work in the field of AI.\n", response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 28, 'total_tokens': 82, 'completion_time': 0.111580355, 'prompt_time': 0.00265645, 'queue_time': None, 'total_time': 0.11423680500000001}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bed19a4c-7b27-4196-ba58-d590162dd53c-0', usage_metadata={'input_tokens': 28, 'output_tokens': 54, 'total_tokens': 82}), HumanMessage(content="What's my name?"), AIMessage(content='Your name is Sahu. 😊  \n\nYou told me at the beginning of our conversation!  \n\nIs there anything else I can help you with?\n'

Error in callback coroutine: ValueError()


"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"